In [1]:
import numpy as np
import pandas as pd

## 第四章 分组

## 分组

### 分组

分组是数据处理中的重要部分, 通过把特定类型的数据进行分组, 来实现更加高效的数据处理.

#### 分组的一般模式

分组操作需要确定三个要素: 分组依据, 数据来源, 操作以及返回结果

使用 `df.groupby()` 函数实现分组.

#### groupby 对象

- 通过 ngroup 属性, 可以得到分组个数
- 通过 groups 属性, 可以返回 组名 映射到 组索引列表 的字典
- 通过 get_group 属性, 可以获取所在组对应的行
- 同样 groupby 对象还可以使用 DataFrame 的属性方法, 如 size 等

分组同样具有几种常用的操作, 下面就要分别介绍相应的 `agg`, `transform` 和 `filter` 函数及其操作.

### 聚合函数

#### 内置聚合函数

内置聚合函数和 DF 数据格式的数据类似, 是将列表中数据统计, 从而从一行或一列中得到一个数值的函数. 常用的统计函数如 

`max/min/mean/median/count/all/any/idxmax/idxmin/mad/nunique/skew/quantile/sum/std/var/sem/size/prod`

都属于聚合函数.

#### agg 方法

虽然在`groupby`对象上定义了许多方便的函数，但仍然有以下不便之处：

* 无法同时使用多个函数
* 无法对特定的列使用特定的聚合函数
* 无法使用自定义的聚合函数
* 无法直接对结果的列名在聚合前进行自定义命名

通过 agg 方法可以实现这些功能

- 使用多个函数

	当使用多个聚合函数时，需要用列表的形式把内置聚合函数对应的字符串传入，先前提到的所有字符串都是合法的.
	e.g. `gb.agg(['sum', 'idxmax', 'skew'])`

- 对特定的列使用特定的聚合函数

	可以通过使用列表的方式进行描述, 可以指定列进行函数操作
	e.g. `gb.agg({'Height':['mean','max'], 'Weight':'count'})`

- 使用自定义的聚合函数

	只需要在自定义函数的时候保证函数返回的是标量值, 就可以引用自定义函数
	这里需要注意, 自定义函数引用时不需要加引号

- 聚合结果重命名

	将上述函数的位置改成 tuple, 其中第一个元素会成为新的名字, 第二个元素为聚合函数的名称, 既可以使用内置聚合函数也可以使用自定义函数

### 变换和过滤

#### transform 方法

变换函数的返回值为同长度的序列, 最常用的内置变换函数是累计函数: `cumcount/cumsum/cumprod/cummax/cummin`, 它们的使用方式和聚合函数类似, 只不过完成的是组内累计操作.

使用 transform 方法可以实现自定义的变换函数, 这个函数要求传入数据源的序列, 返回和传入数据一致长度的 DataFrame 结构.

在 transform 中的函数返回值为一个标量时也可以运行, 此时会触发标量广播机制. 这种机制在特征工程中很常用, 比如构造两列新特征来分别表示样本所在性别组的身高均值和体重均值.

#### 组索引和过滤

在之前索引的知识点中, 索引一直都是作为行筛选的方法, 不会对组进行筛选. 因此相对应的, 现在就在这里使用 filter 方法实现组的过滤, 过滤的原理和索引一致, 对数据进行要求, 不符合要求的组不显示.

`gb.filter(lambda x: x.shape[0] > 100).head()`

### 跨列分组

#### apply 方法

使用之前提到过的 apply 方法, 可以将自定义函数的功能实现, 此时可以在自定义中自定义一个使用多列数值的聚合函数, 也可以实现跨列分组功能.

#### 内置方法

在`groupby`对象中还定义了`cov`和`corr`函数，从概念上说也属于跨列的分组处理。请利用之前定义的`gb`对象，使用apply函数实现与`gb.cov()`同样的功能并比较它们的性能。


## 练习
### Ex1：汽车数据集
现有一份汽车数据集，其中`Brand, Disp., HP`分别代表汽车品牌、发动机蓄量、发动机输出。

In [2]:
df = pd.read_csv('./data/car.csv')
df.head(3)

,Brand,Price,Country,Reliability,Mileage,Type,Weight,Disp.,HP
0,Eagle Summit 4,8895,USA,4.0,33,Small,2560,97,113
1,Ford Escort 4,7402,USA,2.0,33,Small,2345,114,90
2,Ford Festiva 4,6319,Korea,4.0,37,Small,1845,81,63


1. 先过滤出所属`Country`数超过2个的汽车，即若该汽车的`Country`在总体数据集中出现次数不超过2则剔除，再按`Country`分组计算价格均值、价格变异系数、该`Country`的汽车数量，其中变异系数的计算方法是标准差除以均值，并在结果中把变异系数重命名为`CoV`。
2. 按照表中位置的前三分之一、中间三分之一和后三分之一分组，统计`Price`的均值。
3. 对类型`Type`分组，对`Price`和`HP`分别计算最大值和最小值，结果会产生多级索引，请用下划线把多级列索引合并为单层索引。
4. 对类型`Type`分组，对`HP`进行组内的`min-max`归一化。
5. 对类型`Type`分组，计算`Disp.`与`HP`的相关系数。

In [15]:
df_1 = df.groupby('Country').filter(lambda x: x.shape[0] > 2).groupby('Country')['Price'].agg([('Cov', lambda x: x.std()/x.mean()), 'mean', 'count'])
df_1.head()

,Cov,mean,count
Country,,,
Japan,0.387429,13938.052632,19
Japan/USA,0.240040,10067.571429,7
Korea,0.243435,7857.333333,3
USA,0.203344,12543.269231,26


In [16]:
condition = ['Head']*20+['Mid']*20+['Tail']*20
df.groupby(condition)['Price'].mean()

Head     9069.95
Mid     13356.40
Tail    15420.65
Name: Price, dtype: float64

In [18]:
df_2 = df.groupby('Type').agg({'Price': ['max'], 'HP': ['min']})
df_2.columns = df_2.columns.map(lambda x:'_'.join(x))
df_2

,Price_max,HP_min
Type,,
Compact,18900,95
Large,17257,150
Medium,24760,110
Small,9995,63
Sporty,13945,92
Van,15395,106


In [21]:
df.groupby('Type')['HP'].transform(lambda x: (x - x.min())/(x.max()-x.min()))

0     1.000000
1     0.540000
2     0.000000
3     0.580000
4     0.800000
5     0.380000
6     0.540000
7     0.220000
8     0.540000
9     0.200000
10    0.780000
11    0.300000
12    0.740000
13    0.586466
14    0.060150
15    1.000000
16    0.135338
17    0.120301
18    0.360902
19    0.360902
20    0.000000
21    0.037594
22    0.276596
23    0.319149
24    0.000000
25    0.978723
26    0.063830
27    0.638298
28    0.319149
29    0.148936
30    1.000000
31    0.914894
32    0.319149
33    0.531915
34    0.744681
35    0.425532
36    0.404255
37    0.625000
38    0.000000
39    0.500000
40    0.462500
41    0.500000
42    0.375000
43    0.375000
44    0.000000
45    0.600000
46    0.625000
47    0.000000
48    0.312500
49    1.000000
50    0.750000
51    1.000000
52    0.000000
53    0.090909
54    1.000000
55    0.886364
56    1.000000
57    0.022727
58    0.727273
59    0.000000
Name: HP, dtype: float64

In [22]:
df.groupby('Type')['HP', 'Disp.'].apply(lambda x:np.corrcoef(x['HP'].values, x['Disp.'].values)[0,1])

C:\Users\wjie\AppData\Local\Temp\ipykernel_41468\3602552587.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df.groupby('Type')['HP', 'Disp.'].apply(lambda x:np.corrcoef(x['HP'].values, x['Disp.'].values)[0,1])


Type
Compact    0.586087
Large     -0.242765
Medium     0.370491
Small      0.603916
Sporty     0.871426
Van        0.819881
dtype: float64

### Ex2：实现transform函数
* `groupby`对象的构造方法是`my_groupby(df, group_cols)`
* 支持单列分组与多列分组
* 支持带有标量广播的`my_groupby(df)[col].transform(my_func)`功能
* `pandas`的`transform`不能跨列计算，请支持此功能，即仍返回`Series`但`col`参数为多列
* 无需考虑性能与异常处理，只需实现上述功能，在给出测试样例的同时与`pandas`中的`transform`对比结果是否一致

In [ ]:
class my_groupby:
    
    def __init__(self, df, group_cols) -> None:
        self.df = df.copy()
        self.group_cols = df.columns.tolist()
        
        
    